### Goal:Create a Research Agent that can read a blog, extract product names, get product details from Bedrock Knowledge Base (vector store).

Blog: https://aws.amazon.com/blogs/machine-learning/securely-launch-and-scale-your-agents-and-tools-on-amazon-bedrock-agentcore-runtime/<br>
Hint: Use http_search tool in strands agents to read content<br><br>
Entity Extraction: Date, Authors, Central Idea, AWS Products, Adjective Count, and Adjectives used :-) <br>
Hint: Use a good prompt to return information in dictionary format<br><br>
Extract product information using Bedrock Knowledgebase. Use its ID.<br>
Hint: Use retrieve tool in strands agents<br><br>

In [ ]:
# Install the required modules
!pip install strands-agents --quiet
!pip install strands-agents-tools --quiet

In [ ]:
#Let's write an agent that will extract entities from the blog.
#We will later use this agent as a tool in another agent.

from strands import Agent, tool
from strands_tools import http_request
from strands.models.bedrock import BedrockModel

@tool
def extract_blog_info(blog_url: str) -> str:
    """Extract content from the provided URL in a dictionary format

    Args:
        blog_url (str): A single URL string to extract content from.

    Returns:
        str: A formatted string containing the extracted content from the URL
    """

    # Specify the LLM to be used by the agent
    bedrock_model = BedrockModel(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        region_name="us-west-2",
        temperature=0.1,  # Low temperature for consistent structured output
    )

    # Create a system prompt that defines the Agent behavior
    system_prompt="""
    Use the http_request tool to fetch content from the provided URL.
    Extract the following information from the content and return ONLY valid JSON:

    {
        "authors": list of authors,
        "publication_date": "YYYY-MM-DD",
        "central_idea": "brief summary of main topic",
        "aws_services": list of aws products,
        "security_topics": list of security related topics,
        "performance_topics": list of performance related topics,
        "cost_topics": list of cost topics,
        "customer_names": list of customer names,
        "links_to_other_blogs": list of links to other blogs,
        "code_snippet_count": number of code snippets,
        "qualitative_word_count": number of words that refer to quality such as high, low, good but do not quantify,
        "quantitative_phrases": list of phrases that quantify such as 10 user queries,
        "compete_mentions": list of words that refer to competitors of AWS
    }

    Rules:
    - Return only the JSON object, no explanatory text
    - If information is not found, use null for strings/arrays or 0 for counts
    - Extract unique adjectives from the main content, not navigation/footer text
    - Include only unique adjectives in lowercase
    """

    # Create the Agent object
    # The http_request tool will read the contents of the URL that will be provided in prompt.
    blog_agent = Agent(model = bedrock_model, tools=[http_request], system_prompt=system_prompt)

    #The prompt to read the AWS blog
    blog_prompt = f"Extract relevant information from the URL: {blog_url}"

    #invoke the agent
    blog_extract = blog_agent(blog_prompt)
    return blog_extract

# extract_blog_info("https://aws.amazon.com/blogs/machine-learning/securely-launch-and-scale-your-agents-and-tools-on-amazon-bedrock-agentcore-runtime/")

In [ ]:
blog_extract = extract_blog_info("https://aws.amazon.com/blogs/machine-learning/securely-launch-and-scale-your-agents-and-tools-on-amazon-bedrock-agentcore-runtime/")

In [ ]:
# Assume the file kb_id.txt has your bedrock Knowledge Base ID stored.
# Assume Bedrock Knowledgebase has already indexed the product documentation.

kb_id = ""
with open('kb_id.txt', 'r') as f:
    kb_id = f.read().strip()

In [ ]:
# If you dont want the agent to seek consent from users for each action, set the following to true.
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"

In [ ]:
from strands import Agent, tool
from strands.models.bedrock import BedrockModel
from strands_tools import retrieve, file_write

# Specify the model to use and a low temperatures so the LLM is less imaginative.
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.2
)

system_prompt = f"""
To answer the question, stick to the facts that you extract from the URL and the Bedrock Knowledgebase with ID {kb_id}.
"""

# retrieve is a tool that will query Bedrock KB. 
# extract_blog_info is an agent we created earlier. It's being used as a tool here.
research_agent = Agent(
    tools=[retrieve, extract_blog_info, file_write],
    model=bedrock_model,
    system_prompt=system_prompt
)

blog_url = f"https://aws.amazon.com/blogs/machine-learning/securely-launch-and-scale-your-agents-and-tools-on-amazon-bedrock-agentcore-runtime/"

query_text = f"""
Read the blog {blog_url}.
Explain what each of the product mentioned in the blog does, in 75 words, by extracting information from Bedrock Knowledgebase.
Write the final research to a file name product_research.md
"""

#Invoke the agent
research_response = research_agent(query_text)

In [ ]:
print("=== Agent Result ===")
print(f"Stop Reason: {research_response.stop_reason}")
print(f"\nMessage Content:")
print(research_response.message['content'][0]['text'])

In [ ]:
def print_agent_metrics(result):
    print(f"\n=== Metrics ===")
    print(f"Cycle Count: {result.metrics.cycle_count}")
    print(f"Total Tokens: {result.metrics.accumulated_usage['totalTokens']:,d}")
    print(f"Input Tokens: {result.metrics.accumulated_usage['inputTokens']:,d}; Assuming $3 per M input tokens, cost = ${(result.metrics.accumulated_usage['inputTokens']/1000000)*3:,.4f}")
    print(f"Output Tokens: {result.metrics.accumulated_usage['outputTokens']:,d}; Assuming $15 per M output tokens, cost = ${(result.metrics.accumulated_usage['outputTokens']/1000000)*15:,.4f}")
    m_costs = f"If there were 1,000 such research tasks per month, the monthly costs would be"
    i_costs = (result.metrics.accumulated_usage['inputTokens']/1000000)*3*1000 # input token costs
    o_costs = (result.metrics.accumulated_usage['outputTokens']/1000000)*15*1000 #output token costs]
    t_costs = i_costs + o_costs
    print(f"{m_costs}: ${t_costs:,.2f}")
    print(f"\nLatency: {result.metrics.accumulated_metrics['latencyMs']:,d}ms")
    
    print(f"\n=== Tool Usage ===")
    for tool_name, metrics in result.metrics.tool_metrics.items():
        print(f"- {tool_name}: {metrics.call_count} calls, {metrics.success_count} successful, {metrics.error_count} errors, {metrics.total_time:,.2f} seconds")

# Usage
print_agent_metrics(research_response)

In [ ]:
# Demonstrate that you can ask a follow up question.
followup_response = research_agent("How many services were mentioned that were related to AgentCore?")